<a href="https://colab.research.google.com/github/jhlee93/AFCompetition/blob/main/2368/task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 안녕하세요. AIFactory입니다 ☺️
이 노트북은 제출을 위한 가이드라인 노트북입니다.  

Colab에서 작업 하실 경우, 반드시 구글 드라이브에 사본을 저장을(파일 -> Drive에 사본 저장) 하신 후에 사용해주세요.
  
이번 태스크는 스코어 경쟁을 통해 입상자를 선별하는 대회가 아닌,  
AIF의 **추론 자동화**를 경험해보고 이 과정에서 불편한 점, 개선 되었으면 하는 점 등 여러분들의 다양한 의견을 듣기 위해서이니 다양한 의견을 공유해 주시면 감사드리겠습니다.



# 주의 사항

### Colab 환경에서 작업 하시는 경우
- Colab 노트북 공유 권한을 ***공유 -> 링크가 있는 모든 사용자*** 로 설정해주세요.
- 구글 드라이브 마운트는 하지 말아주세요.
    - AIF 추론 환경에서는 Colab 전용 라이브러리를 사용 할 수 없으며,
    - 참가자분의 개인 구글 드라이브에 접근 할 수 없습니다.
</br></br>

### 개인 로컬 환경에서 작업 하시는 경우
- 제출 스크립트 파일 이름을 반드시 ***task.ipynb***로 해주세요.
</br></br>

### 공통
- 제출 스크립트에 학습 코드를 포함하지 말아주세요. 학습 코드는 별도 작업 공간에서 진행해주세요.
- 현재 작업중인 경로에서 모델 추론에 불필요한 파일(학습 데이터 등...)이 제출되지 않도록 주의해주세요.
- 모든 경로는 현재 작업중인 스크립트 파일을 기준으로 **상대 경로**로 설정해주세요.
- 이번 베타 테스트의 추론 환경은 CPU만 사용하여 평가를 진행하니 참고하여 코드를 작성해주세요.
- AIF의 추론 환경은 Colab과 완전히 동일하지 않으니, 추론 환경에 설치된 패키지 항목을 참고하신 후 필요한 패키지를 설치해주세요.

</br></br>
**추론 환경에 설치된 패키지 리스트**
```
tensorflow==2.11.1
keras-cv
torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1
albumentations
transformers
scikit-learn
numpy
Cython
pycocotools
pandas
jupyter
notebook
matplotlib
seaborn
plotly
Pillow
opencv-python
scikit-image
```

# 필요 패키지 설치

In [ ]:
# aifactory 패키지는 반드시 설치해주세요!
!pip install -U aifactory

In [ ]:
# 필요한 패키지를 추가로 설치해주세요.
!pip install gdown
!pip install transformers
!pip install konlpy
# ...

In [ ]:
# 패키지 임포트
import os
import random
import numpy as np
import torch
import tensorflow as tf
import transformers
# ...

In [ ]:
# 무작위성 통제
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
# ...

# 모델 로드 함수를 정의해주세요.
- 모델 로드 함수는 참가자 분들께서 이미 학습 완료한 모델 가중치 파일을 읽어와 메모리에 올려 놓은 후 반환 되어야 합니다.
- 모델 가중치 파일을 현재 노트북 파일 기준 ***동일 경로 또는 하위 경로***에 있어야하며, ***상대 경로***로 설정 되어야 합니다.
- Colab에서 작업 하시는 경우 가중치 파일을 수동으로 업로드 하시거나 또는,  
개인 구글 드라이브에 업로드 후(***공유 권한: 링크가 있는 모든 사용자***) `gdown`을 통해 다운 받아 사용하세요.
- 이 함수가 받는 인자(Arguments)는 없거나, default 값으로 설정 되어야 합니다.
- 함수 명칭은 자유롭게 정의해주세요.


```python
# Example 1
import gdown

def tf_load_model():
    weights_path = './my_weights.h5'
    gdown.downalod("GoolgeDriveSharedLink", weights_path, fuzzy=True)
    ...
    model.load_weights(weights_path)
    return model

# Example 2
def torch_load_model(weights_path='./my_weights.h5'):
    ...
    model.load_state_dict(torch.load(weights_path))
    model.eval()
    return model
```


In [ ]:
def load_model():
    # 모델 아키텍쳐
    classification_model = Sequential([
        layers.LSTM(...)
        ...
        layers.Dense(1, )
    ])

    # 모델 가중치 로드
    classification_model.load_weights('./my_weights.h5')

    # ...
    tokenizer = Tokenizer()

    return classification_model, tokenizer

# 예측 함수를 정의해주세요.
- 예측 함수는 반드시 **2개의 인자**를 다음과 같이 순서대로 받아야합니다.
    - 첫 번째 인자 - AIF가 제공하는 평가 입력 데이터(X_test)
    - 두 번째 인자 - 가중치를 읽어온 모델

- 첫 번째 인자(X_test)는 ***평가용 영화 리뷰 글***들의 경로가 담긴 ***List***를 전달합니다.  
    - `X_test = ['너무 웃겨서 제대로 못봄', '극장에서 상영하면 죽여버린다', '재밌는 코미디 영화네요^^', ... ]`
    - X_test가 제공되는 형식을 참고하여 예측 및 전/후 처리를 통해 **출력 규칙**에 맞게 결과를 반환해주세요.
    
- **출력 규칙**
    - X_test 전체 예측값을 1(긍정)과 0(부정)으로 구성된 ***List***를 반환해주세요.
    `[1, 0, 1, 0, 1, ...]`



In [ ]:
# 예측 함수는 AIF에서 제공하는 X_test의 입력 정보와, 메모리에 올라간 사전 학습 모델을 입력 받습니다.
def predict(X_test, model):
    result = []
    for text in X_test:
        pred = model.predict(text)
        ...
        result.append(pred)

    # result = [1, 0, 1, 0, ...]
    return result


# submit 함수를 정의해 주세요.  
- 이 함수의 명칭은 반드시 **submit** 으로 해주세요.
- **submit** 함수는 AIF가 참가자분들이 정의한 2개의 함수를 전달받아 실행하기 위해 사용됩니다.
- **submit** 함수를 통해 위에서정의한 함수를 **1.모델 로드 함수**와 **2.예측 함수** 순으로 반환해 주세요.
- AIF가 submit 함수를 통해 채점하는 방식은 다음과 같으니 참고해주세요.  
    ```python
    # AIF 채점 코드 예시
    X_test = ['너무 웃겨서 제대로 못봄', '극장에서 상영하면 죽여버린다', '재밌는 코미디 영화네요^^', ... ]
    y_test = [1, 0, 1, ...]

    load_model, predict = submit()
    models = load_model() # 참가팀 모델 로드 함수 실행
    y_pred = predict(X_test, models) # 참가팀 예측 함수 실행

    from sklearn.metrics import accuracy_score
    score = accuracy_score(y_test, y_pred)
    ```

In [ ]:

def submit():
    return load_model, predict

# 제출하기
- aifactory 라이브러리를 통해 submit 함수를 제출합니다.
- aif.submit 함수는 반드시 `if __name__ == "__main__":` 아래에 작성해주세요.
- aifactory.score.submit 함수는 3개의 인자를 받습니다.
    - ***model_name*** - 제출시 리더보드에 기록하고자 하는 문구
    - ***key*** - 대회 참가자에게 제공되는 TASK_KEY  
    - ***func*** - submit 함수

- TASK_KEY 확인 방법
    - https://aifactory.space/competition/detail/2367 페이지 접속
    - 우측 상단 `내정보` -> `태스크 정보` 클릭
    - 1분야 : 이미지 모델 퀘스트에서 `사용자 키 보기` 클릭

In [ ]:
import aifactory.score as aif

if __name__ == "__main__":
    aif.submit(
        model_name="MyFirstSubmission",
        key='TASK_KEY',
        func=submit)